In [0]:
'''
# Testing for GPU availability
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))'''

### **To avoid tedious debates with the TA. Dina, we have to print all outputs of "each" step regardless of the importance of the step.** @Zyad


---











In [0]:
.'''
# Installing database file/s
!wget http://104.197.136.14/ds/2016.04/2016.04C.multisnr.pkl.bz2
!bzip2 -d 2016.04C.multisnr.pkl.bz2
!ls

In [0]:
'''
# The classes Sequential and Dense are used to specify the nodes, connections, specifications of the neural network.
import keras
from keras.models import Sequential
from keras.layers import Dense

In [0]:
'''
import pickle
import gzip
import numpy

with open('2016.04C.multisnr.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    p = u.load()

In [0]:
'''
#Kareem
#alternative method (like paper code)
import numpy as np
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], p.keys())))), [1,0])
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(p[(mod,snr)])
        for i in range(p[(mod,snr)].shape[0]):  lbl.append((mod,snr))
X = np.vstack(X)
print(X.shape)
print("one sample :")
print(X[1])

In [0]:
'''
#Kareem
#all keys
print("keys:")
print(p.keys())
#print(p["QPSK",2].shape)
print("shape of all data:")
print(X.shape)
#list of all shapes:
shapes=[]
snrs=[]
mod_types=[]
for x in p.keys():
  shapes.append(p[x].shape)
  snrs.append(x[1])
  mod_types.append(x[0])
print("same distribution of number of samples as shown:")
print(sorted(shapes)[0:20])
print(sorted(shapes)[20:40])
print(sorted(shapes)[40:60])

print("same modulation type but different SNRs:")
print(p["QPSK",2].shape)
print(p["QPSK",6].shape)
print(p["QPSK",8].shape)
print(p["QPSK",-4].shape)
print(p["QPSK",18].shape)
print(p["QPSK",-6].shape)
print(p["QPSK",12].shape)


print("Again : same modulation type but different SNRs:")
print(p["AM-DSB",2].shape)
print(p["AM-DSB",6].shape)
print(p["AM-DSB",8].shape)
print(p["AM-DSB",-4].shape)
print(p["AM-DSB",18].shape)
print(p["AM-DSB",-6].shape)
print(p["AM-DSB",12].shape)

print("same SNR for Different modulation types:")
print(p["QPSK",2].shape)
print(p["PAM4",2].shape)
print(p["AM-DSB",2].shape)
print(p["QAM64",2].shape)
print(p["AM-SSB",2].shape)
print(p["QAM16",2].shape)
print(p["8PSK",2].shape)
print("values in keys:")
print(set(snrs))
print(len(set(snrs)))
print(set(mod_types))
print(len(set(mod_types)))


# Conclusion of dataset structure:

Dataset has 11 modulation types and 20 SNR values forming 220 different keys
each key has different number of samples

each Modulation type (ex : QPSK,8PSK ...etc) has same number of samples for different SNRs



In [0]:
'''
sum=0
for x in p.values():
  sum+=x.shape[0]
print(sum)
# for key,value in p.items():
#   if key[0]=='PAM4':
#     sum+=value.shape[0]
  
# print(sum)

In [0]:
'''
print(type(p))
data = {}
allm = []
for k in p.keys():
  data[k[0]] = {}
  allm.append(k[0])
mod = sorted(set(allm))
print(mod)
print(data)
print(len(allm))


In [0]:
'''
import numpy as np
from numpy import zeros, newaxis

for m in mod:
  for k in p.keys():
    if k[0] == m :
      for sig in range(len(p[k])):
        a = np.array(p[k][sig][0])[:, newaxis]
        b = np.array(p[k][sig][1])[:, newaxis]
        if k[1] not in data[k[0]]:
          data[k[0]][k[1]] = []
        data[k[0]][k[1]].append([a,b])

2 dimentions vectors because complex values a vector for real value and imaginary value

In [0]:
'''
dat = []
#Training data and labels 
X = []
Y = []
#dictionary contating samples for each snr, key is snr and value is list of samples having this snr
samples_per_snr = {}
#dictonary accessed by snr and gives labels of mod types ,snr is index
labels_ofSamples_per_snr = {}
#dictionary saving all one-hot encodes of different modulation
mval = {}
count = 0

#changed variables z one_hot_label, x -> samples_per_snr , y->labels_ofSamples_per_snr  both used for testing

#loop all modulation techniques ex QPSK,BSK...etc
for m in mod:
  one_hot_label = np.zeros((len(mod),))
  one_hot_label[count] = 1     
  mval[m] = one_hot_label
  #repeat for each snr for modulation technique m
  for snr in data[m]:
    #print(m,snr)
    dat = data[m][snr]
    #train with 2/3 of data from start
    #1/3 for each channel ??
    for d in dat[:int(len(dat)//1.5)]:
      X.append(d)
      Y.append(one_hot_label)
      #loop the remaining 1/3 of data to test with them
      for d in dat[int(len(dat)//1.5):]:
        #only performed once to set the 20 snrs as keys in dictionary
        #each value for key is list of samples having this snrs
        if not snr in samples_per_snr:
          samples_per_snr[snr] = []
          labels_ofSamples_per_snr[snr] = []
        #the real loop body :
        samples_per_snr[snr].append(d)
        labels_ofSamples_per_snr[snr].append(one_hot_label)
  #increment loop counter
  count += 1 


In [0]:
'''
print(len(X))
print(len(Y))

In [0]:
# Making the Confusion Matrix
# from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(..., ...) 

In [0]:
'''
Sample Test


input_dim_ = 30 #@param {type:"integer"}

# Initialising the ANN
classifier = Sequential()

# Initialising input layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = input_dim_))

# Adding the second hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

**Mahinour**

In [0]:
!pip install keras 
!pip install tflearn
!pip install tensor

In [0]:
%matplotlib inline
import os,random
#os.environ["KERAS_BACKEND"] = "theano"
os.environ["KERAS_BACKEND"] = "tensorflow"
#os.environ["THEANO_FLAGS"]  = "device=cuda,floatX=float32"
import numpy as np
from keras.utils import np_utils
import keras.models as models
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import *
from keras.optimizers import adam
import matplotlib.pyplot as plt
import seaborn as sns
import pickle, random, sys, keras
# Installing database file/s
!wget http://104.197.136.14/ds/2016.04/2016.04C.multisnr.pkl.bz2
!bzip2 -d 2016.04C.multisnr.pkl.bz2
!ls
# The classes Sequential and Dense are used to specify the nodes, connections, specifications of the neural network.

with open('2016.04C.multisnr.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    Xd = u.load()
    


In [0]:
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
X = np.vstack(X)

In [0]:
# Partition the data
#  into training and test sets of the form we can train/test on 
#  while keeping SNR and Mod labels handy for each
np.random.seed(2016)
n_examples = X.shape[0]
n_train = n_examples * 0.5
train_idx = np.random.choice(range(0,n_examples), size= int(n_train), replace=False)
test_idx = list(set(range(0,n_examples))-set(train_idx))
X_train = X[train_idx]
X_test =  X[test_idx]

In [0]:
def to_onehot(yy):
    yy1 = np.zeros([len(yy), max(yy)+1])
    yy1[np.arange(len(yy)),yy] = 1
    return yy1 
Y_train = to_onehot(list(map(lambda x:0 mods.index(lbl[x][0]), train_idx)))
Y_test = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), test_idx)))

In [0]:
in_shp = list(X_train.shape[1:])
print (X_train.shape, in_shp)
classes = mods

In [0]:
import keras.models as models
# Build VT-CNN2 Neural Net model using Keras primitives -- 
#  - Reshape [N,2,128] to [N,1,2,128] on input
#  - Pass through 2 2DConv/ReLu layers
#  - Pass through 2 Dense layers (ReLu and Softmax)
#  - Perform categorical cross entropy optimization
 
dr = 0.5 # dropout rate (%)
model = keras.models.Sequential()
model.add(Reshape([1]+in_shp, input_shape=in_shp))
model.add(ZeroPadding2D((0, 2)))
model.add(Conv2D(256, (1, 3),padding='valid', activation="relu", name="conv1",kernel_initializer='glorot_uniform',data_format="channels_first"))
model.add(Dropout(dr))
model.add(ZeroPadding2D((0, 2)))
model.add(Conv2D(80,(2, 3), padding='valid', activation="relu", name="conv2", kernel_initializer='glorot_uniform',data_format="channels_first"))
model.add(Dropout(dr))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_normal', name="dense1"))
model.add(Dropout(dr))
model.add(Dense( len(classes), kernel_initializer='he_normal', name="dense2" ))
model.add(Activation('softmax'))
model.add(Reshape([len(classes)]))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [0]:
# Set up some params 
nb_epoch = 25    # number of epochs to train on
batch_size = 500  # training batch size

In [0]:
# perform training ...
#   - call the main training loop in keras for our network+dataset
filepath = 'convmodrecnets_CNN2_0.5.wts.h5'
model.fit(X_train,
    Y_train,
    batch_size=batch_size,
    epochs=nb_epoch,
    verbose=2,
     validation_split=0.3,
    callbacks = [
        keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
    ])
# we re-load the best weights once training is finished
model.load_weights(filepath)

In [0]:
scscore = model.evaluate(X_test, Y_test, batch_size=batch_size)
print(model.metrics_names)
print (score)